# Treaps!

A treap is a binary tree in which each node has a key and a priority. The treap is a BST with respect to its keys and a max heap with respect to its priorities. Treaps can be used to implement randomized binary search trees, which self-balance without the need for complex rules.

In [7]:
class TreapNode:
    """A node in a treap.
    
    Attributes
    ----------
    key
        The node's key, used to place it in the BST.
    priority
        Node's priority, use to place it in the heap.
    parent : Optional[TreapNode]
        The node's parent. If this is a root node, this is None.
    left : Optional[TreapNode]
        The node's left child; if there is none, this is None.
    right : Optional[TreapNode]
        The node's right child; if there is non, this is None.
    
    """
    
    def __init__(self, key, priority):
        self.key = key
        self.priority = priority
        self.parent = None
        self.left = None
        self.right = None
        
    def __repr__(self):
        """Nicely displays the node."""
        return f'{self.__class__.__name__}(key={self.key}, priority={self.priority})'
    
    def is_leaf(self):
        """Returns True if this node has no children, else False."""
        return self.left is None and self.right is None 
    

class Treap:
    """Half heap, half binary search tree. It's a treap!"""
    
    def __init__(self):
        """Create an empty treap."""
        self.root = None
                
    def delete(self, x: TreapNode):
        """Delete the node from the treap.
        
        Parameters
        ----------
        x : TreapNode
            The node to delete. Note that this is a TreapNode object,
            not a key. If you wish to delete a node with a specific
            key, you should query to find its node.
            
        """
        # rotate the node down until it becomes a leaf
        while not x.is_leaf():
            if x.left is not None and x.right is not None:
                if x.left.priority > x.right.priority:
                    self._right_rotate(x)
                else:
                    self._left_rotate(x)
            elif x.left is not None:
                self._right_rotate(x)
            elif x.right is not None:
                self._left_rotate(x)
        
        # the node is now a leaf and can be removed. this
        # is done by removing the reference from the node's parent
        # to this node
        p = x.parent
        if p is not None:
            if x is p.left:
                p.left = None
            else:
                p.right = None
                
    def query(self, target):
        """Return the TreapNode with the specific key.
        
        Parameters
        ----------
        target
            The key to look for.
            
        Returns
        -------
        TreapNode
            The node with the specific key. Assumes that keys are unique.
            
        """
        # walk down the tree, starting at root, searching for key
        current_node = self.root
        while current_node is not None:
            if current_node.key == target:
                return current_node
            elif current_node.key < target:
                current_node = current_node.right
            else:
                current_node = current_node.left
        return None
        
    def insert(self, key, priority):
        """Create a new node with given key and priority.
        
        Parameters
        ----------
        new_key
            The node's new key. Should be unique.
        new_priority
            The node's priority. Need not be unique.
        
        Raises
        ------
        ValueError
            If the new node's key is already in the treap.
            
        """
        current_node = self.root
        parent = None
        
        # walk down the tree in search of the place to put the new key
        while current_node is not None:
            parent = current_node
            if current_node.key == key:
                raise ValueError(f'Duplicate key "{key}" not allowed.')
            if current_node.key < key:
                current_node = current_node.right
            elif current_node.key > key:
                current_node = current_node.left
                
        # create the new node
        new_node = TreapNode(key=key, priority=priority)
        new_node.parent = parent
        
        # place it in the tree
        if parent is None:
            self.root = new_node
        elif parent.key < key:
            parent.right = new_node
        else:
            parent.left = new_node

        # the heap invariant may be broken -- rotate the node up until
        # it is once again satisfied
        while new_node != self.root and new_node.priority > new_node.parent.priority:
            if new_node.parent.left is new_node:
                self._right_rotate(new_node.parent)
            else:
                self._left_rotate(new_node.parent)
            
    def _right_rotate(self, x: TreapNode):
        """Rotate x down to the right."""
        u = x.left
        B = u.right
        C = x.right
        p = x.parent
        
        x.left = B
        if B is not None: B.parent = x
        
        u.right = x
        x.parent = u
            
        u.parent = p
        
        if p is None:
            self.root = u
        elif p.left is x:
            p.left = u
        else:
            p.right = u
            
    def _left_rotate(self, x: TreapNode):
        """Rotate x down to the left."""
        u = x.right
        A = u.left
        C = x.left
        p = x.parent
                
        x.right = A
        if A is not None: A.parent = x
        
        u.left = x
        x.parent = u
            
        u.parent = p
        
        if p is None:
            self.root = u
        elif p.left is x:
            p.left = u
        else:
            p.right = u

In [10]:
treap = Treap()
treap.insert(42, priority=10)
treap.insert(50, priority=20)
treap.insert(3, priority=99)
treap.insert(10, priority=5)

In [11]:
treap.query(42)

TreapNode(key=42, priority=10)

In [13]:
treap.query(100)